<a href="https://colab.research.google.com/github/wushengl/PGM-project-sound-event-detection/blob/main/YOHO_upgraded_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Install and Import Packages

In [ ]:
!git clone https://github.com/DemisEom/SpecAugment.git
!pip install /content/SpecAugment/
!pip install tensorflow-addons
!pip install tensorflow-io
!pip install sed_eval


Cloning into 'SpecAugment'...
remote: Enumerating objects: 271, done.
remote: Total 271 (delta 0), reused 0 (delta 0), pack-reused 271
Receiving objects: 100% (271/271), 429.45 KiB | 8.59 MiB/s, done.
Resolving deltas: 100% (150/150), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./SpecAugment
  Preparing metadata (setup.py) ... done
  Created wheel for SpecAugment: filename=SpecAugment-1.2.3-py3-none-any.whl size=16695 sha256=b4b1bd327e393c56b6476182a7123d792a2b5f54ecce5ca443150c2de73eda28
  Stored in directory: /tmp/pip-ephem-wheel-cache-afpnaah6/wheels/2a/3f/22/b1010812c6f4e7022aa188158ac89f91f71028dbda2f35e740
Successfully built SpecAugment
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 9.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple

In [ ]:
import os
import math
import time
import csv
import random
import os.path
import pdb

import re
import glob
import shutil
import librosa
import keras
import sed_eval
import dcase_util
import pickle

import soundfile as sf
import numpy as np
import tensorflow_io as tfio
import tensorflow as tf

from google.colab import drive
from zipfile import ZipFile
from subprocess import Popen, PIPE
from os.path import dirname
from tensorflow.keras import regularizers
from keras.regularizers import l2
from SpecAugment import spec_augment_tensorflow

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


# 2. Download Dataset and Preprocessing 

## 2.1 Preparing .npy dataset and Google drive 

- downloaded dataset are preprocessed log melspectrograms 
  - data size: (1001,40)
  - label size: (32,30)
- To download the original Urban-SED dataset by Salamon et al., visit this [page](http://urbansed.weebly.com/). Ensure the dataset is saved into /content/Urban-SED-V2.tar.gz. (**We've already downloaded the original dataset and uploaded to shared drive**)

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=19vTnCCNrfh1mUsVzTBKQEDpjpHwveTXQ' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=19vTnCCNrfh1mUsVzTBKQEDpjpHwveTXQ" -O train-data.zip && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-0SZavmwJI7zB8734_uek0ejLUTaa1M7' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-0SZavmwJI7zB8734_uek0ejLUTaa1M7" -O val-data.zip && rm -rf /tmp/cookies.txt

--2023-04-27 18:05:23--  https://docs.google.com/uc?export=download&confirm=t&id=19vTnCCNrfh1mUsVzTBKQEDpjpHwveTXQ
Resolving docs.google.com (docs.google.com)... 173.194.210.139, 173.194.210.113, 173.194.210.102, ...
Connecting to docs.google.com (docs.google.com)|173.194.210.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-6k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mh3rff2kltddbjm8ivb4kgmnkitsacv6/1682618700000/04739181468756608208/*/19vTnCCNrfh1mUsVzTBKQEDpjpHwveTXQ?e=download&uuid=999b51e4-98a0-4170-973a-5041f65304ac [following]
--2023-04-27 18:05:23--  https://doc-08-6k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mh3rff2kltddbjm8ivb4kgmnkitsacv6/1682618700000/04739181468756608208/*/19vTnCCNrfh1mUsVzTBKQEDpjpHwveTXQ?e=download&uuid=999b51e4-98a0-4170-973a-5041f65304ac
Resolving doc-08-6k-docs.googleusercontent.com (doc-08-6k-docs.googleusercontent.com)... 173.194.212.

In [ ]:
# extract zip files for both train and validation data 
zip_name0 = "/content/train-data.zip"
with ZipFile(zip_name0, 'r') as zip:
  zip.extractall()

zip_name1 = "/content/val-data.zip"
with ZipFile(zip_name1, 'r') as zip:
  zip.extractall()

In [ ]:
print(len(glob.glob("/content/content/train-data/*.npy")))
print(len(glob.glob("/content/content/train-data/label-*.npy")))
print(len(glob.glob("/content/content/val-data/*.npy")))
print(len(glob.glob("/content/content/val-data/label-*.npy")))


12000
6000
4000
2000


## 2.2 Helper functions

In [ ]:
# Function for constructing list of audio frames and window ranges from original dataset audio file 
def construct_examples(audio_path, win_len = 2.56, hop_len = 1.0, sr = 44100.0):

  win_len_t = win_len
  hop_len_t = hop_len

  win_len = int(sr*win_len)
  hop_len = int(sr*hop_len)

  a, sr = sf.read(audio_path)

  if a.shape[0] < win_len:
    a_padded = np.zeros((win_len, ))
    a_padded[0:a.shape[0]] = a  

  else:
    no_of_hops = math.ceil((a.shape[0] - win_len) / hop_len)
    a_padded = np.zeros((int(win_len + hop_len*no_of_hops), ))
    a_padded[0:a.shape[0]] = a  

  a_ex = [a_padded[i - win_len : i] for i in range(win_len, a_padded.shape[0]+1, hop_len)]
  print([i - win_len for i in range(win_len, a_padded.shape[0]+1, hop_len)])
  win_ranges = [((i - win_len)/sr, i/sr) for i in range(win_len, a_padded.shape[0]+1, hop_len)]

  return a_ex, win_ranges

In [ ]:
def get_log_melspectrogram(audio, sr = 44100, hop_length = 441, win_length = 1764, n_fft = 2048, n_mels = 40, fmin = 0, fmax = 22050):
    """Return the log-scaled Mel bands of an audio signal."""
    audio_2 = librosa.util.normalize(audio)
    bands = librosa.feature.melspectrogram(
        y=audio_2, sr=sr, hop_length=hop_length, win_length = win_length, n_fft=n_fft, n_mels=n_mels)
    return librosa.core.power_to_db(bands)

In [ ]:
class_dict = {'air_conditioner': 0,
        'car_horn': 1,
        'children_playing': 2,
        'dog_bark': 3,
        'drilling': 4,
        'engine_idling': 5,
        'gun_shot': 6,
        'jackhammer': 7,
        'siren': 8,
        'street_music': 9}

In [ ]:
# Function for reading events from txt files in original dataset label file 
def read_annotation(filename):
    events = []
    with open(filename, 'r') as csvfile:
        spamreader = csv.reader(csvfile, delimiter='\t', quotechar='|')
        for row in spamreader:
            events.append(row)
    return events
    
# Test function
# events = read_annotation("/content/Urban-SED/URBAN-SED_v2.0.0/annotations/train/soundscape_train_bimodal0.txt")
# events


In [ ]:
def construct_labels(annotation_path, win_start, win_end, win_len):
  events = read_annotation(annotation_path)

  ann = [[float(e[0]), float(e[1]), e[2]] for e in events]

  curr_ann = []

  for a in ann:
    if a[1] > win_start and a[0] <= win_end: 
    # if a[0] >= win_start and a[0] < win_end:
      curr_start = max(a[0] - win_start, 0.0)
      curr_end = min(a[1] - win_start, win_len)
      curr_ann.append([curr_start, curr_end, a[2]])    

  class_set = set([c[2] for c in curr_ann])
  class_wise_events = {}

  for c in list(class_set):
    class_wise_events[c] = []


  for c in curr_ann:
    class_wise_events[c[2]].append(c)
    
  max_event_silence = 0.0
  all_events = []

  for k in list(class_wise_events.keys()):
    curr_events = class_wise_events[k]
    count = 0

    while count < len(curr_events) - 1:
      if (curr_events[count][1] >= curr_events[count + 1][0]) or (curr_events[count + 1][0] - curr_events[count][1] <= max_event_silence):
        curr_events[count][1] = max(curr_events[count + 1][1], curr_events[count][1])
        del curr_events[count + 1]
      else:
        count += 1

    all_events += curr_events

  for i in range(len(all_events)):
    all_events[i][0] = round(all_events[i][0], 3)
    all_events[i][1] = round(all_events[i][1], 3)

  all_events.sort(key=lambda x: x[0])

  return all_events

In [ ]:
def to_seg_by_class(events, class_dict, hop_len = 441, n_frames = 1001, sr=44100):
  # events = smoothe_events(events)
  labels = np.zeros((n_frames, 10), dtype=np.float32)

  for e in events:
    t1 = float(e[0])
    t1 = int(t1 / hop_len * sr)
    t2 = float(e[1])
    t2 = int(t2 / hop_len * sr)

    labels[t1:t2, class_dict[e[2]]] = 1    
  
  return labels 

In [ ]:
def get_universal_labels(events, class_dict, ex_length = 10.0, no_of_div = 32):
  win_length = ex_length/no_of_div
  labels = np.zeros((no_of_div, len(class_dict.keys()) * 3))
  
  for e in events:

    start_time = float(e[0])
    stop_time = float(e[1])

    start_bin = int(start_time // win_length)
    stop_bin = int(stop_time // win_length)

    start_time_2 = start_time - start_bin * win_length
    stop_time_2 = stop_time - stop_bin * win_length

    n_bins = stop_bin - start_bin

    if n_bins == 0:
      labels[start_bin, class_dict[e[2]] * 3:class_dict[e[2]] * 3 + 3] = [1, start_time_2, stop_time_2]    

    elif n_bins == 1:
      labels[start_bin, class_dict[e[2]] * 3:class_dict[e[2]] * 3 + 3] = [1, start_time_2, win_length]

      if stop_time_2 > 0.0:
        labels[stop_bin, class_dict[e[2]] * 3:class_dict[e[2]] * 3 + 3] = [1, 0.0, stop_time_2]

    elif n_bins > 1:
      labels[start_bin, class_dict[e[2]] * 3:class_dict[e[2]] * 3 + 3] = [1, start_time_2, win_length]

      for i in range(1, n_bins):
        labels[start_bin + i, class_dict[e[2]] * 3:class_dict[e[2]] * 3 + 3] = [1, 0.0, win_length]

      if stop_time_2 > 0.0:
        labels[stop_bin, class_dict[e[2]] * 3:class_dict[e[2]] * 3 + 3] = [1, 0.0, stop_time_2]

  # labels[:, [1, 2, 4, 5]] /= win_length

  for i in range(len(labels)):
    for j in range(len(labels[i])):
      if j % 3 != 0:
        labels[i][j] /= win_length

  return labels

## 2.3 Preparing the dataset from the URBAN-SED website

In [ ]:
#os.mkdir("/content/Urban-SED/")
# unzipping the original dataset, if need to run, make sure the paths are correct and the folders exist 
zip_name = "/content/drive/MyDrive/10708 - Project/code/URBAN-SED_v2.0.0.tar.gz"
shutil.unpack_archive(zip_name, "/content/Urban-SED/")

In [ ]:
# make file list for train and validation dataset from original dataset 

train_files = glob.glob("/content/Urban-SED/URBAN-SED_v2.0.0/audio/train/*.wav")
val_files = glob.glob("/content/Urban-SED/URBAN-SED_v2.0.0/audio/validate/*.wav")

train_ann = glob.glob("/content/Urban-SED/URBAN-SED_v2.0.0/annotations/train/*.txt")
val_ann = glob.glob("/content/Urban-SED/URBAN-SED_v2.0.0/annotations/validate/*.txt")

# if not os.path.exists("/content/original-data/"):
#   os.mkdir("/content/original-data/")
# test the files
print(len(train_files))# = 6000
print(len(train_ann))# = 6000
# sampling rate: 44100
# sample length: 10s

6000
6000


In [ ]:
train_files[0]

'/content/Urban-SED/URBAN-SED_v2.0.0/audio/train/soundscape_train_bimodal954.wav'

In [ ]:
zzz = np.load("/content/content/train-data/ex-0.npy")
print(zzz.shape)

(1001, 40)


# 3. Dataloader: creating the data-label pair

## 3.1 Helper functions

In [ ]:
def tryint(s):
    try:
        return int(s)
    except ValueError:
        return s
    
def alphanum_key(s):
    """ Turn a string into a list of string and number chunks.
        "z23a" -> ["z", 23, "a"]
    """
    return [ tryint(c) for c in re.split('([0-9]+)', s) ]

def sort_nicely(l):
    """ Sort the given list in the way that humans expect.
    """
    l.sort(key=alphanum_key)

## 3.2 Loading .npy datasets into partitions
- load data from saved mel-spectrogram and labels
- save into "partition" which contains both training and validation data
- len(partition['train']) = 6000
- len(partition['validation']) = 4000

In [ ]:
"""
Load the individual numpy arrays into partition
"""
data = glob.glob("/content/content/train-data/ex-*.npy") # + glob.glob("/content/train data/MuSpeak/content/Mel Files/**/mel-id-[0-9]*.npy", recursive=True) 
#data = glob.glob("/content/train data/MuSpeak/content/Mel Files/**/mel-id-[0-9]*.npy", recursive=True) 
sort_nicely(data)

labels = glob.glob("/content/content/train-data/label-*.npy") #+ glob.glob("/content/train data/MuSpeak/content/Mel Files/**/mel-id-label-[0-9]*.npy", recursive=True)
#labels = glob.glob("/content/train data/MuSpeak/content/Mel Files/**/mel-id-label-[0-9]*.npy", recursive=True)
sort_nicely(labels)

train_examples = [(data[i], labels[i]) for i in range(len(data))]

random.seed(4)
random.shuffle(train_examples)

In [ ]:
"""
This loads data for the validation set.
"""
data = glob.glob("/content/content/val-data/ex-*.npy")
sort_nicely(data)

labels = glob.glob("/content/content/val-data/label-*.npy")
sort_nicely(labels)

validation_examples = [(data[i], labels[i]) for i in range(len(data))]

random.seed(4)
random.shuffle(validation_examples)
print(validation_examples[0])

# m = len(test_examples)
# m_validation = 1024
# m_test = 512
# m_train = m - m_validation - m_test

('/content/content/val-data/ex-1211.npy', '/content/content/val-data/label-1211.npy')


In [ ]:
"""
Creating the train & validation partition.
"""
partition = {}
partition['train'] = train_examples
random.shuffle(partition['train'])
partition['validation'] = validation_examples

## 3.3 DataGenerator #1: y size (32,30)

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_examples, batch_size=128, epoch_size = 16384, dim=(1, ),
                 n_classes=2, shuffle=True):
        'Initialization'
        print("Constructor called!!!")
        self.dim = dim
        self.batch_size = batch_size
        self.epoch_size = epoch_size
        self.list_examples = list_examples
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        #print("The self.list_examples is {}".format(self.list_examples))
        return int(np.floor(len(self.list_examples) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_examples[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y
        
    def on_epoch_end(self):
      self.indexes = np.arange(len(self.list_examples))
      if self.shuffle == True:
          np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        # 'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # # Initialization
        #X = np.empty([self.batch_size, 1001, 40, 1], dtype=np.float64)
        X = np.empty([self.batch_size, 1001, 40], dtype=np.float64)
        y = np.empty([self.batch_size, 32, 30], dtype=np.float64) # 1001,10

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
          # Store sample

          xx = np.load(ID[0])

          # v = xx.shape[0]
          # tau = xx.shape[1]

          # xx = tf.expand_dims(xx, axis=0)
          # xx = tf.expand_dims(xx, axis=-1)

          # # warped_mel_spectrogram = sparse_warp(mel_spectrogram)

          # warped_frequency_spectrogram = spec_augment_tensorflow.frequency_masking(xx, v=v, frequency_masking_para=8, frequency_mask_num=1)

          # warped_frequency_time_sepctrogram = spec_augment_tensorflow.time_masking(warped_frequency_spectrogram, tau=tau, time_masking_para=25, time_mask_num=2)

          

          # xx = tf.expand_dims(xx, axis=0)
          # xx = tf.expand_dims(xx, axis=-1)



          # freq_mask = tfio.audio.freq_mask(xx, param=8)

          # time_mask = tfio.audio.time_mask(freq_mask, param=25)

          # xx = tf.squeeze(warped_frequency_time_sepctrogram)
          X[i, :, :] = xx

          # Store class
          yy = np.load(ID[1])
          # yy2 = yy[:, [1, 2, 4, 5]]

          #import pdb;pdb.set_trace()
          y[i, :, :] = yy



        # tau = X.shape[1]          
        # v = X.shape[2]

        # warped_frequency_spectrogram = spec_augment_tensorflow.frequency_masking(X, v=v,  frequency_masking_para=8, frequency_mask_num=1)
        # warped_frequency_time_sepctrogram = spec_augment_tensorflow.time_masking(warped_frequency_spectrogram, tau=tau, time_masking_para=25, time_mask_num=3)

        # X = warped_frequency_time_sepctrogram

        #pdb.set_trace()

        return X, y

# 4. Define Loss function, Binary accuracy and Callback function

## 4.1 Define Loss function and binary accuracy

In [ ]:
def coherence_loss(y_true, y_pred):
  #pdb.set_trace()
  # Calculate the squared difference between consecutive predictions
  y_pred_shifted = tf.roll(y_pred, shift=-1, axis=1) #z_t-1
  coherence_diff = tf.square(y_pred[:, :-1, :] - y_pred_shifted[:, :-1, :]) # [:, :-1, :]
  
  # we can tune this weight
  coherence_weight = 0.1

  #alpha1 different max
  a1 = -0.1
  #alpha2 same min
  a2 = 0.03

  # Create labels for y_true
  #pdb.set_trace()
  shape = tf.shape(y_true)
  y_true_presentlabel = tf.strided_slice(y_true, [0, 0, 0], [shape[0], shape[1], shape[2]], [1, 1, 3])
  y_true_label = tf.tile(y_true_presentlabel,[1,1,3])

  #y_true_label = tf.zeros((tf.shape(y_true)[0], tf.shape(y_true)[1], tf.shape(y_true)[2] // 3))
  #y_true_label = tf.zeros((32, 32, 10))
  #y_true_label = tf.tensor_scatter_nd_update(y_true_label, tf.constant([[0, 0, 0]]), y_true[:, 0, 0:30:3])
  #y_true_label = tf.repeat(y_true_label, 3, axis=-1)

  # Shift y_true_label
  y_true_label_shifted = tf.roll(y_true_label, shift=-1, axis=1)

  # Create indices for max and min differences
  ind_max = tf.cast(tf.not_equal(y_true_label[:, :-1, :], y_true_label_shifted[:, :-1, :]), tf.float32) # [:, :-1, :]
  ind_min = tf.cast(tf.equal(y_true_label[:, :-1, :], y_true_label_shifted[:, :-1, :]), tf.float32) # [:, :-1, :]

  # Compute coherence loss
  L_coh = a1 * ind_max * coherence_diff + a2 * ind_min * coherence_diff

  # Sum the coherence loss along the sequence axis
  return coherence_weight * tf.reduce_sum(L_coh, axis=[-1, -2])

In [ ]:
def my_loss_fn(y_true, y_pred):
  weight = tf.constant([1.0])

  squared_difference = tf.square(y_true - y_pred)

  # squared_difference[:, 1] =  squared_difference[:, 1] * y_true[:, 0]

  # squared_difference[:, 3] =  squared_difference[:, 3] * y_true[:, 2]

  ss_True = squared_difference[:, :, 0] * 0 + 1

  ss_0 = y_true[:, :, 0]
  ss_1 = y_true[:, :, 3]
  ss_2 = y_true[:, :, 6]
  ss_3 = y_true[:, :, 9]
  ss_4 = y_true[:, :, 12]
  ss_5 = y_true[:, :, 15]
  ss_6 = y_true[:, :, 18]
  ss_7 = y_true[:, :, 21]
  ss_8 = y_true[:, :, 24]
  ss_9 = y_true[:, :, 27]

  sss = tf.stack((ss_True, ss_0, ss_0,
                  ss_True, ss_1, ss_1,
                  ss_True, ss_2, ss_2,
                  ss_True, ss_3, ss_3,
                  ss_True, ss_4, ss_4,
                  ss_True, ss_5, ss_5,
                  ss_True, ss_5, ss_6,
                  ss_True, ss_5, ss_7,
                  ss_True, ss_5, ss_8,
                  ss_True, ss_5, ss_9), axis = 2)
  
  squared_difference =  tf.multiply(squared_difference, sss)

  # mask_1 = tf.cast(tf.equal(ss1, 0), dtype = np.float32)
  # mask_1 = mask_1 * weight

  # mask_2 = tf.cast(tf.equal(ss1, 1), dtype = np.float32)

  # mask_3 = tf.cast(tf.equal(ss4, 0), dtype = np.float32)
  # mask_3 = mask_3 * weight

  # mask_4 = tf.cast(tf.equal(ss4, 1), dtype = np.float32)

  # mask_s = mask_1 + mask_2

  # mask_m = mask_3 + mask_4

  # mmm = tf.stack((mask_s, ss0, ss0, mask_m, ss3, ss3), axis = 2)

  # squared_difference =  tf.multiply(squared_difference, mmm)

  # Add coherence loss to your existing loss
  total_loss = tf.reduce_sum(squared_difference, axis=[-1, -2]) + coherence_loss(y_true, y_pred)

  return total_loss #tf.reduce_sum(squared_difference, axis=[-1, -2])  # Note the `axis=-1`

In [ ]:
def binary_acc(y_true, y_pred):
  threshold = tf.constant([0.5])

  binary_true = tf.stack((y_true[:, :, 0], y_true[:, :, 3], y_true[:, :, 6], y_true[:, :, 9], y_true[:, :, 12],
                          y_true[:, :, 15], y_true[:, :, 18], y_true[:, :, 21], y_true[:, :, 24], y_true[:, :, 27]), axis=1)
  binary_pred = tf.stack((y_pred[:, :, 0], y_pred[:, :, 3], y_pred[:, :, 6], y_pred[:, :, 9], y_pred[:, :, 12]
                          , y_pred[:, :, 15], y_pred[:, :, 18], y_pred[:, :, 21], y_pred[:, :, 24], y_pred[:, :, 27]), axis=1)

  binary_true = tf.greater_equal(binary_true, threshold)
  binary_pred = tf.greater_equal(binary_pred, threshold)

  # binary_pred = tf.greater(y_pred[:, [0, 2]], threshold)

  # acc = tf.square(y_true - y_pred)
  acc = tf.cast((binary_true == binary_pred), tf.float32)

  return tf.reduce_mean(acc, axis=[-1, -2])  # Note the `axis=-1`

## 4.2 Callback functions

callback is a set of functions that are applied at various stages of the training process. Callbacks can be used to perform a wide range of tasks during training, such as:

- Saving the model or model weights at specified intervals.
- Stopping the training process early based on a specific metric or condition.
- Adjusting the learning rate of the optimizer during training.
- Logging metrics or other information during training.

Customized Callback Included in this block:

- MyCustomCallback_3: report only loss
- MyCustomCallback_44: report loss, f-measure loss, error rate
  - Need mk_preds_YOHO_mel_2
  - save to "/content/eval-files-2/"
- MyCustomCallback_CRNN

In [ ]:
# This optimises val loss for Wave-U-Net YOHO
# Back to Val Binary acc.
class MyCustomCallback_3(tf.keras.callbacks.Callback):
  def __init__(self, model_dir, patience=0):
    super(MyCustomCallback_3, self).__init__()
    self.patience = patience
    # best_weights to store the weights at which the minimum loss occurs.
    self.best_weights = None
    self.model_best_path = os.path.join(model_dir, 'model-best.h5')
    self.model_last_path = os.path.join(model_dir, 'model-last-epoch.h5')
    self.custom_params = {"best_loss":np.inf, "last_epoch":0}
    
    self.custom_params_path = os.path.join(model_dir, 'custom_params.pickle')
    if os.path.isfile(self.custom_params_path):
      with open(self.custom_params_path, 'rb') as f:
        self.custom_params = pickle.load(f)
      # best_model = tf.keras.models.load_model(self.model_best_path, custom_objects={'my_loss_fn':my_loss_fn, 
      #             'binary_acc':binary_acc, 'TCN':TCN(), 'SpeechF1':SpeechF1(), 'MusicF1':MusicF1(),
      #             'OnAcc':OnAcc(), 'OnOffAcc':OnOffAcc(),'AudioClipLayer':AudioClipLayer(), 
      #             'InterpolationLayer':InterpolationLayer(), 'CropLayer':CropLayer(),
      #             'IndependentOutputLayer':IndependentOutputLayer(), 'DiffOutputLayer':DiffOutputLayer()})
      # best_model = tf.keras.models.clone_model(self.model)
      # best_model.load_weights(self.model_best_path)
      # self.best_weights = best_model.get_weights()


  def on_train_begin(self, logs=None):
    # The number of epoch it has waited when loss is no longer minimum.
    self.wait = 0
    # The epoch the training stops at.
    self.stopped_epoch = 0
    # Initialize the best F1 as 0.0.
    self.is_impatient = False

  def on_train_end(self, logs=None):
    if not self.is_impatient:
      print("Restoring model weights from the end of the best epoch.")
      self.model.set_weights(self.best_weights)
      # temp_model_path = self.model_path.replace(".h5", "_temp.h5")
      #os.remove(temp_model_path)

  def on_epoch_end(self, epoch, logs=None):
    current_val_loss = logs.get("val_loss")
    self.model.save_weights(self.model_last_path)
    self.custom_params["last_epoch"] = self.custom_params["last_epoch"] + 1

    if current_val_loss < self.custom_params['best_loss']:
      self.custom_params['best_loss'] = current_val_loss
      self.wait = 0
      self.best_weights = self.model.get_weights()
      self.model.save_weights(self.model_best_path)

    else:
        self.wait += 1
        if self.wait >= self.patience:
            self.stopped_epoch = epoch
            self.is_impatient = True
            self.model.stop_training = True
            print("Restoring model weights from the end of the best epoch.")
            self.model.set_weights(self.best_weights)
            #os.remove(temp_model_path)
    with open(self.custom_params_path, 'wb') as f:
      pickle.dump(self.custom_params, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
class MyCustomCallback_44(tf.keras.callbacks.Callback):
  def __init__(self):
    super(MyCustomCallback_44, self).__init__()
    self.best_f1 = 0.0
    self.best_error = np.inf

    
  def on_train_begin(self, logs=None):
    pass

  def on_train_end(self, logs=None):
    pass

  def on_epoch_end(self, epoch, logs=None):
    if True:
      start_time = time.time()

      # for ii, audio in enumerate(val_files):
      #   audio_file_path = audio
      #   see = mk_preds_YOHO_mel(self.model, ii)
      #   n_label = n_label = "/content/eval-files-2/" + os.path.basename(audio_file_path).replace(".wav" ,"") + "-se-prediction.txt"

      #   with open(n_label, 'w') as fp:
      #     fp.write('\n'.join('{},{},{}'.format(round(x[0], 5), round(x[1], 5), x[2]) for x in see))

      mk_preds_YOHO_mel_2(self.model, mss_ins=mss_ins_2)

      stop_time = time.time()
      # print("Time for prediction: {}".format(stop_time - start_time))

      start_time = time.time()
      destination = "/content/eval-files-2/"
      test_set = glob.glob(destination + "*[0-9].txt")

      eval_path = "/content/"


      file_list = [
          {
          'reference_file': tt,
          'estimated_file': tt.replace(".txt","-se-prediction.txt")
          }
          for tt in test_set
      ]

      data = []

      # Get used event labels
      all_data = dcase_util.containers.MetaDataContainer()
      for file_pair in file_list:
          reference_event_list = sed_eval.io.load_event_list(
              filename=file_pair['reference_file']
          )
          estimated_event_list = sed_eval.io.load_event_list(
              filename=file_pair['estimated_file']
          )

          data.append({'reference_event_list': reference_event_list,
                      'estimated_event_list': estimated_event_list})

          all_data += reference_event_list

      event_labels = all_data.unique_event_labels

      # Start evaluating
      # Create metrics classes, define parameters
      segment_based_metrics = sed_eval.sound_event.SegmentBasedMetrics(
          event_label_list=event_labels,
          time_resolution=1.0
      )

      event_based_metrics = sed_eval.sound_event.EventBasedMetrics(
          event_label_list=event_labels,
          t_collar=1.0
      )

      # Go through files
      for file_pair in data:
          segment_based_metrics.evaluate(
              reference_event_list=file_pair['reference_event_list'],
              estimated_event_list=file_pair['estimated_event_list']
          )

          event_based_metrics.evaluate(
              reference_event_list=file_pair['reference_event_list'],
              estimated_event_list=file_pair['estimated_event_list']
          )

      # Get only certain metrics
      overall_segment_based_metrics = segment_based_metrics.results_overall_metrics()
      curr_f1 = overall_segment_based_metrics['f_measure']['f_measure']
      curr_error = overall_segment_based_metrics['error_rate']['error_rate']

      if curr_f1 > self.best_f1:
        self.best_f1 = curr_f1
        self.model.save_weights("/content/drive/MyDrive/model-best-f1.h5")

      if curr_error < self.best_error:
        self.best_error = curr_error
        self.model.save_weights("/content/drive/MyDrive/model-best-error.h5")

      print("F-measure: {:.3f} vs {:.3f}".format(curr_f1, self.best_f1))
      print("Error rate: {:.3f} vs {:.3f}".format(curr_error, self.best_error))

      stop_time = time.time()

      # print("Time to calculate metrics: {}".format(stop_time - start_time))

      # Or print all metrics as reports

## 4.3 Callback44 helper function: mk_preds_YOHO_mel_2 

### Initialize mss_in

In [ ]:
#rev class dict
rev_class_dict = ['air_conditioner', 'car_horn', 'children_playing',
          'dog_bark', 'drilling', 'engine_idling', 'gun_shot',
          'jackhammer','siren', 'street_music']

# Creates mel spctrograms for training

win_length = 10.0
hop_size = 10.0
mss_ins = []
win_ranges_list = []
n_classes = 10

for ii, audio in enumerate(val_files):
  a, win_ranges = construct_examples(audio, win_len=win_length,hop_len=hop_size)

  mss_in = np.zeros((1, 1001, 40))

  # if len(a) > 1:
  #   print(audio)

  preds = np.zeros((len(a), 3, 30)) # I think this is an unused variable


  for i in range(1):
    M = get_log_melspectrogram(a[i])
    mss_in[i, :, :] = M.T
  mss_ins.append(mss_in)
  win_ranges_list.append(win_ranges)

#test
a, sr = sf.read("/content/Urban-SED/URBAN-SED_v2.0.0/audio/validate/soundscape_validate_bimodal528.wav")
a.shape[0]/sr
len(mss_ins) #2000
mss_ins_2 = np.concatenate(mss_ins, axis=0)
mss_ins_2.shape #2000,1001,40

[0]
[0]
[0]
[0]
[0]
[0, 441000]
[0]
[0]
[0]
[0]
[0]
[0, 441000]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0, 441000]
[0]
[0]
[0, 441000]
[0]
[0]
[0, 441000]
[0, 441000]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0, 441000]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0, 441000]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0, 441000]
[0, 441000]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0, 441000]
[0, 441000]
[0]
[0]
[0]
[0]
[0, 441000]
[0]
[0]
[0]
[0, 441000]
[0]
[0]
[0, 441000]
[0]
[0]
[0]
[0]
[0, 441000]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0, 441000]
[0]
[0, 441000]
[0]
[0]
[0]
[0]
[0]
[0, 441000]
[0, 441000]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0, 441000]
[0, 441000]
[0]
[0]
[0]
[0, 441000]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0, 441000]
[0, 441000]
[0]
[0, 441000]
[0]
[0]
[0]
[0, 441000]
[0]
[0]
[0]
[0]
[0, 441000]
[0, 441000]
[0, 441000]
[0]
[0]
[0, 441000]
[0]
[0]
[0]
[0]
[0, 441000]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0, 441000]
[0]
[0]
[0]
[0]
[0, 

(2000, 1001, 40)

In [ ]:
def extract_labels_2(annotation_path):
  events = read_annotation(annotation_path)

  ann = [[float(e[0]), float(e[1]), e[2]] for e in events]
  
  n_label = "/content/eval-files-2/" + os.path.basename(annotation_path)

  with open(n_label, 'w') as fp:
    fp.write('\n'.join('{},{},{}'.format(round(x[0], 5), round(x[1], 5), x[2]) for x in ann))

In [ ]:
if not os.path.exists("/content/eval-files-2/"):
  os.mkdir("/content/eval-files-2/")
for audio in val_files:
  extract_labels_2(audio.replace(".wav", ".txt").replace("audio", "annotations"))

### Helper Function

In [ ]:
def mk_preds_YOHO_mel_2(model, mss_ins=mss_ins_2, no_of_div = 32, hop_size = 10.0, discard = 0.3, win_length = 10.0, max_event_silence = 0.3, sampling_rate = 44100):
  preds = model.predict(mss_ins_2)

  for ii, audio in enumerate(val_files):
    audio_file_path = audio

    events = []
    p = preds[ii, :, :]
    events_curr = []
    win_width = win_length / no_of_div
    for j in range(len(p)):
      for jjj in range(0, n_classes):
        if p[j][jjj*3] >= 0.5:
          start = win_width * j + win_width * p[j][jjj*3+1] + win_ranges_list[ii][0][0]
          end = p[j][jjj*3+2] * win_width + start
          events_curr.append([start, end, rev_class_dict[jjj]])

    events += events_curr


    class_set = set([c[2] for c in events])
    class_wise_events = {}

    for c in list(class_set):
      class_wise_events[c] = []


    for c in events:
      class_wise_events[c[2]].append(c)
      
    
    all_events = []

    for k in list(class_wise_events.keys()):
      curr_events = class_wise_events[k]
      count = 0

      while count < len(curr_events) - 1:
        if (curr_events[count][1] >= curr_events[count + 1][0]) or (curr_events[count + 1][0] - curr_events[count][1] <= max_event_silence):
          curr_events[count][1] = max(curr_events[count + 1][1], curr_events[count][1])
          del curr_events[count + 1]
        else:
          count += 1

      all_events += curr_events

    for i in range(len(all_events)):
      all_events[i][0] = round(all_events[i][0], 3)
      all_events[i][1] = round(all_events[i][1], 3)

    all_events.sort(key=lambda x: x[0])



    see = all_events
    n_label = n_label = "/content/eval-files-2/" + os.path.basename(audio_file_path).replace(".wav" ,"") + "-se-prediction.txt"

    with open(n_label, 'w') as fp:
      fp.write('\n'.join('{},{},{}'.format(round(x[0], 5), round(x[1], 5), x[2]) for x in see))

In [ ]:
def frames_to_time(f, sr = 44100.0, hop_size = 441):
  return f * hop_size / sr

def preds_to_se(p, win_start, audio_clip_length = 2.56):
  start_dicts = [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]
  stop_dicts = [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100]


  start_speech = -100
  start_music = -100
  stop_speech = -100
  stop_music = -100

  audio_events = []

  n_frames = p.shape[0]

  for j in range(p.shape[1]):
    if p[0, j] >= 0.5:
      start_dicts[j] = 0

  for j in range(p.shape[1]):
    for i in range(n_frames - 1):
      if p[i, j] < 0.5 and p[i+1, j] >= 0.5:
        start_dicts[j] = i+1

      elif p[i, j] >= 0.5 and p[i + 1, j] < 0.5:
        stop_dicts[j] = i
        start_time = frames_to_time(start_dicts[j])
        stop_time = frames_to_time(stop_dicts[j])

        audio_events.append([start_time+win_start, stop_time+win_start, rev_class_dict[j]])
        start_dicts[j] = -100
        stop_dicts[j] = -100

    if start_dicts[j] != -100:
      start_time = frames_to_time(start_dicts[j])
      stop_time = audio_clip_length
      audio_events.append([start_time+win_start, stop_time+win_start, rev_class_dict[j]])
      start_dicts[j] = -100
      stop_dicts[j] = -100

  audio_events.sort(key = lambda x: x[0]) 
  return audio_events

# 5. Define the YamNet Model
- YOHO adapted from MobileNet, modified the final layers
- MobileNet has also been employed for audio classification by  YamNet

In [ ]:
LAYER_DEFS = [
    # (layer_function, kernel, stride, num_filters)
    ([3, 3], 1,   64),
    ([3, 3], 2,  128),
    ([3, 3], 1,  128),
    ([3, 3], 2,  256),
    ([3, 3], 1,  256),
    ([3, 3], 2,  512),
    ([3, 3], 1,  512),
    ([3, 3], 1,  512),
    ([3, 3], 1,  512),
    ([3, 3], 1,  512),
    ([3, 3], 1,  512),
    ([3, 3], 2, 1024),
    ([3, 3], 1, 1024),
    ([3, 3], 1, 512),
    ([3, 3], 1, 256),
    ([3, 3], 1, 128),
    # ([3, 3], 1, 128),
    # ([3, 3], 1, 128)
]

In [ ]:
"""
Manually define YamNet
"""

rbm_layer1_shape = 1001
rbm_layer2_1_shape = 700
rbm_layer2_2_shape = 301

# params = yamnet_params.Params()
m_features = tf.keras.Input(shape=(1001, 40), name="mel_input")

X = tf.keras.layers.Reshape((1001, 40))(m_features)  

#X = m_features

# add linear layer and relu for RBM layer 1, output X is hidden_activation, shape (1001,)
X = tf.keras.layers.Reshape(target_shape=(-1, 1001 * 40))(X)
rbm_feature_1 = tf.keras.layers.Dense(units=rbm_layer1_shape, activation='sigmoid',name='rbm_1')(X) #reshape((-1, 1001 * 40))

# add linear layer and relu for RBM layer 2 
X1 = tf.keras.layers.Dense(units=rbm_layer2_1_shape, activation='sigmoid',name='rbm_2_1')(rbm_feature_1)
X2 = tf.keras.layers.Dense(units=rbm_layer2_2_shape, activation='sigmoid',name='rbm_2_2')(rbm_feature_1)

#X1 = tf.keras.layers.Reshape(target_shape=(rbm_layer2_1_shape,1))(X1)
#X2 = tf.keras.layers.Reshape(target_shape=(rbm_layer2_2_shape,1))(X2)
rbm_feature_2 = tf.concat([X1,X2],axis=2)  

# combine spectrogram and hidden feature, resulting in (1001, 40+n) 
#rbm_feature_1 = tf.expand_dims(rbm_feature_1, axis=-1) 
#bm_feature_2 = tf.expand_dims(rbm_feature_2, axis=-1) 

#rbm_feature_1 = tf.keras.layers.Reshape(target_shape=(1001,1))(rbm_feature_1)
#rbm_feature_2 = tf.keras.layers.Reshape(target_shape=(1001,1))(rbm_feature_2)
rbm_feature_1 = tf.transpose(rbm_feature_1, perm=[0, 2, 1])
rbm_feature_2 = tf.transpose(rbm_feature_2, perm=[0, 2, 1])

concat_feature = tf.concat([m_features,rbm_feature_1,rbm_feature_2],axis=2)  # tf has place holder for batch_size at axis=0

# original CNN network starting from here 

X = tf.keras.layers.Reshape((1001, 42, 1))(concat_feature)  
X = tf.keras.layers.Conv2D(filters = 32, kernel_size=[3, 3], strides=2, padding='same', use_bias=False,
                           activation=None, name = "layer1/conv",
                             kernel_regularizer=l2(1e-3), bias_regularizer=l2(1e-3))(X)
X = tf.keras.layers.BatchNormalization(center=True, scale=False, epsilon=1e-4, name = "layer1/bn")(X)
X = tf.keras.layers.ReLU(name="layer1/relu")(X)

# X = tf.keras.layers.SpatialDropout2D(0.5)(X)  

for i in range(len(LAYER_DEFS)):
  X = tf.keras.layers.DepthwiseConv2D(kernel_size=LAYER_DEFS[i][0], strides = LAYER_DEFS[i][1], depth_multiplier=1, padding='same', use_bias=False,
                                      activation=None, name="layer"+ str(i + 2)+"/depthwise_conv")(X)
  X = tf.keras.layers.BatchNormalization(center=True, scale=False, epsilon=1e-4, name = "layer"+ str(i + 2)+"/depthwise_conv/bn")(X)
  X = tf.keras.layers.ReLU(name="layer"+ str(i + 2)+"/depthwise_conv/relu")(X)
  X = tf.keras.layers.Conv2D(filters = LAYER_DEFS[i][2], kernel_size=[1, 1], strides=1, padding='same', use_bias=False, activation=None,
                             name = "layer"+ str(i + 2)+"/pointwise_conv",
                             kernel_regularizer=l2(1e-2), bias_regularizer=l2(1e-2))(X)
  X = tf.keras.layers.BatchNormalization(center=True, scale=False, epsilon=1e-4, name = "layer"+ str(i + 2)+"/pointwise_conv/bn")(X)
  X = tf.keras.layers.ReLU(name="layer"+ str(i + 2)+"/pointwise_conv/relu")(X)

  X = tf.keras.layers.SpatialDropout2D(0.1)(X)

  # X = tf.keras.layers.Dropout(0.1)(X)


# X = tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides = 1, depth_multiplier=1, padding='same', use_bias=False, activation=None, name="layer2/depthwise_conv")(X)
# X = tf.keras.layers.BatchNormalization(center=True, scale=False, epsilon=1e-4, name = "layer2/depthwise_conv/bn")(X)
# X = tf.keras.layers.ReLU(name="layer2/depthwise_conv/relu")(X)
# X = tf.keras.layers.Conv2D(filters = 64, kernel_size=[1, 1], strides=1, padding='same', use_bias=False, activation=None, name = "layer2/pointwise_conv")(X)
# X = tf.keras.layers.BatchNormalization(center=True, scale=False, epsilon=1e-4, name = "layer2/pointwise_conv/bn")(X)
# X = tf.keras.layers.ReLU(name="layer2/pointwise_conv/relu")(X)

_, _, sx, sy = X.shape
X = tf.keras.layers.Reshape((-1, int(sx * sy)))(X)
# X = tf.keras.layers.Dropout(0.4)(X)

# X = _separable_conv("extra", [3, 3], 2, 1024, params)(predictions)
# X = tf.keras.layers.Lambda(lambda x: tf.squeeze(x, axis=-2))(X)
# X = layers.Dense(2, name="speech_and_music", activation='sigmoid')(embeddings)
# X = tf.keras.layers.Flatten()(X)
pred = tf.keras.layers.Conv1D(30,kernel_size=1, activation="sigmoid")(X)
# pred = tf.keras.layers.Dense(6, activation="sigmoid")(X)
model = tf.keras.Model(
      name='yamnet_frames', inputs=m_features,
      outputs=[pred])

In [ ]:
model.summary()

Model: "yamnet_frames"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 mel_input (InputLayer)         [(None, 1001, 40)]   0           []                               
                                                                                                  
 reshape (Reshape)              (None, 1001, 40)     0           ['mel_input[0][0]']              
                                                                                                  
 reshape_1 (Reshape)            (None, 1, 40040)     0           ['reshape[0][0]']                
                                                                                                  
 rbm_1 (Dense)                  (None, 1, 1001)      40081041    ['reshape_1[0][0]']              
                                                                                      

In [ ]:
# model.load_weights("/content/drive/MyDrive/10708 - Project/YOHO-music-speech.h5") # model-best-error-YamNet.h5

### Training based on previously saved model

In [ ]:
# run this to load weights for RBM layers 

rbm_layer_1 = model.get_layer('rbm_1') 
rbm_layer_2_1 = model.get_layer('rbm_2_1')
rbm_layer_2_2 = model.get_layer('rbm_2_2')


rbm_1_weights = np.load('/content/drive/MyDrive/10708 - Project/code/saved models/rbm_variables_layer1.npz') 
rbm_2_1_weights = np.load('/content/drive/MyDrive/10708 - Project/code/saved models/rbm_variables_layer2_700.npz')
rbm_2_2_weights = np.load('/content/drive/MyDrive/10708 - Project/code/saved models/rbm_variables_layer2_301.npz')


In [ ]:
rbm_layer_1.set_weights([rbm_1_weights['W'], rbm_1_weights['hb']])
rbm_layer_2_1.set_weights([rbm_2_1_weights['W'], rbm_2_1_weights['hb']])
rbm_layer_2_2.set_weights([rbm_2_2_weights['W'], rbm_2_2_weights['hb']])

In [ ]:
# run this to load model weights without RBM parts

model.load_weights("/content/drive/MyDrive/10708 - Project/code/saved models/model-best-f1-coh_loss-0d5-rbm2.h5")

## CDBN [NO NEED TO RUN]
 Pre-trained weights have been saved to saved-models in shared drive. Just keep these blocks updated for future use is fine. 

In [ ]:
import numpy as np
import tensorflow as tf

class RBM:
    def __init__(self, visible_units, hidden_units, learning_rate=0.01, batch_size=10):
        self.visible_units = visible_units
        self.hidden_units = hidden_units
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        
        # Initialize weights and biases
        self.W = tf.Variable(tf.random.normal([visible_units, hidden_units], mean=0, stddev=0.01), name="weights")
        self.vb = tf.Variable(tf.zeros([visible_units]), name="visible_bias")
        self.hb = tf.Variable(tf.zeros([hidden_units]), name="hidden_bias")

        self.W = tf.cast(self.W,dtype=tf.float64)
        self.vb = tf.cast(self.vb,dtype=tf.float64)
        self.hb = tf.cast(self.hb,dtype=tf.float64)

        self.W = tf.Variable(self.W)
        self.vb = tf.Variable(self.vb)
        self.hb = tf.Variable(self.hb)

    def sample_hidden(self, visible_probabilities):
        hidden_activations = tf.nn.sigmoid(tf.matmul(visible_probabilities, self.W) + self.hb)
        hidden_samples = tf.nn.relu(tf.sign(hidden_activations - tf.cast(tf.random.uniform(tf.shape(hidden_activations)),dtype=tf.float64)))
        return hidden_samples

    def sample_visible(self, hidden_probabilities):
        visible_activations = tf.nn.sigmoid(tf.matmul(hidden_probabilities, tf.transpose(self.W)) + self.vb)
        visible_samples = tf.nn.relu(tf.sign(visible_activations - tf.cast(tf.random.uniform(tf.shape(visible_activations)),dtype=tf.float64)))
        return visible_samples

    def contrastive_divergence(self, input_data, k=1):
        visible_probabilities = input_data
        hidden_probabilities = self.sample_hidden(visible_probabilities)
        
        for _ in range(k):
            visible_probabilities = self.sample_visible(hidden_probabilities)
            hidden_probabilities = self.sample_hidden(visible_probabilities)

        return visible_probabilities

    def update_weights(self, input_data, k=1):
        positive_hidden_probabilities = self.sample_hidden(input_data)
        negative_visible_probabilities = self.contrastive_divergence(input_data, k=k)

        positive_associations = tf.matmul(tf.transpose(input_data), positive_hidden_probabilities)
        negative_associations = tf.matmul(tf.transpose(negative_visible_probabilities), self.sample_hidden(negative_visible_probabilities))

        self.W.assign_add(self.learning_rate * (positive_associations - negative_associations))
        self.vb.assign_add(self.learning_rate * tf.reduce_mean(input_data - negative_visible_probabilities, 0))
        self.hb.assign_add(self.learning_rate * tf.reduce_mean(positive_hidden_probabilities - self.sample_hidden(negative_visible_probabilities), 0))

    def train(self, input_data, epochs=10, k=1):
        for epoch in range(epochs):
            print(f"Epoch: {epoch+1}/{epochs}")
            flattened_input_data = input_data.reshape(-1, self.visible_units)
            self.update_weights(flattened_input_data, k=k)

    def train_old(self, training_generator, steps_per_epoch, epochs=10, k=1):
        for epoch in range(epochs):
            print(f"Epoch: {epoch+1}/{epochs}")
            for step in range(steps_per_epoch):
                input_data = next(training_generator)
                flattened_input_data = input_data.reshape(-1, self.visible_units)
                self.update_weights(flattened_input_data, k=k)

    def load_weights(self, weight_file, visible_bias_file, hidden_bias_file):
        self.W.assign(np.load(weight_file))
        self.vb.assign(np.load(visible_bias_file))
        self.hb.assign(np.load(hidden_bias_file))




In [ ]:
# input_data = np.random.rand(100, 1001, 40)  # Replace this with your actual input data
# flattened_input_data = input_data.reshape(-1, 1001 * 40)

data = glob.glob("/content/content/train-data/ex-*.npy")
sort_nicely(data)

all_data = np.empty((6000,1001,40))
idx = 0
for file_name in data:
  this_data = np.load(file_name)
  all_data[idx,:,:] = this_data.reshape(-1,1001,40)  #np.concatenate((all_data, this_data.reshape(-1,1001,40)),axis=0)
  idx +=1




In [ ]:
visible_units = 1001 * 40
hidden_units = 1001  # You can adjust this to the desired number of hidden units

# Create an RBM layer
rbm = RBM(visible_units, hidden_units, learning_rate=0.01, batch_size=10)

#steps_per_epoch = 188
# Train the RBM layer
#all_data = tf.constant(all_data,dtype=tf.float64)
#all_data = all_data.numpy()
#data_zzz = all_data[:100,:,:]
rbm.train(all_data.astype("double"), epochs=10, k=1)

Epoch: 1/10
Epoch: 2/10
Epoch: 3/10
Epoch: 4/10
Epoch: 5/10
Epoch: 6/10
Epoch: 7/10
Epoch: 8/10
Epoch: 9/10
Epoch: 10/10


In [ ]:
variables = {'W': rbm.W, 'vb': rbm.vb, 'hb': rbm.hb}
#np.savez('rbm_variables.npz', **variables)
np.savez_compressed('rbm_variables.npz', **variables)

In [ ]:
flattened_input_data = all_data.reshape(-1, 1001 * 40) #all_data = (5000, 1001, 40)
input_hidden = rbm.sample_hidden(flattened_input_data) 

second_visible_units = hidden_units  # The hidden units from the previous RBM layer become the visible units for the next layer
second_hidden_units = 700  # Adjust this to the desired number of hidden units for the second RBM layer

rbm2 = RBM(second_visible_units, second_hidden_units, learning_rate=0.01, batch_size=10)
rbm2.train(input_hidden, epochs=10, k=1)
input_hidden2 = rbm2.sample_hidden(input_hidden)

In [ ]:
variables2 = {'W': rbm2.W, 'vb': rbm2.vb, 'hb': rbm2.hb}
np.savez('/content/drive/MyDrive/10708 - Project/rbm_variables_layer2_700.npz', **variables2)

In [ ]:
flattened_input_data = all_data.reshape(-1, 1001 * 40) #all_data = (5000, 1001, 40)
input_hidden = rbm.sample_hidden(flattened_input_data) 

second_visible_units = hidden_units  # The hidden units from the previous RBM layer become the visible units for the next layer
second_hidden_units = 301  # Adjust this to the desired number of hidden units for the second RBM layer

rbm3 = RBM(second_visible_units, second_hidden_units, learning_rate=0.01, batch_size=10)
rbm3.train(input_hidden, epochs=10, k=1)
input_hidden3 = rbm3.sample_hidden(input_hidden)

In [ ]:
variables3 = {'W': rbm3.W, 'vb': rbm3.vb, 'hb': rbm3.hb}
np.savez('/content/drive/MyDrive/10708 - Project/rbm_variables_layer2_301.npz', **variables3)

# 6. Model Training

Parameter/settings to play with:

- DataGenerator 
  - batch size
  - epoch_size (minibatch? set to 0 for SGD?)
- callback
  - keras callbacks (e.g. tf.keras.callbacks.EarlyStopping)
  - custom callbacks (check previous block for definition)
- optimizer 
  - 2 examples contained in this block, choose the setting we want

In [ ]:
# Parametersa
params = {'dim': (1, ),
      'batch_size': 32,
      'epoch_size': 5,
      'n_classes': 10, # original 2, should be 10? But this seems to be not used in DataGenerator class?
      'shuffle': True}

# Generators
training_generator = DataGenerator(partition['train'], **params)
validation_generator = DataGenerator(partition['validation'], **params)

Constructor called!!!
Constructor called!!!


In [ ]:
#callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)

In [ ]:
m_auc = tf.keras.metrics.AUC(
    num_thresholds=200,
    curve="ROC",
    summation_method="interpolation",
    multi_label=True,
    num_labels=162,
    # label_weights=[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0],
    from_logits=False,
)

In [ ]:
# compile model with optimizer setting 1
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-2,
        decay_steps=250,
        decay_rate=0.9)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
model.compile(optimizer=optimizer, loss=my_loss_fn, metrics=[binary_acc])

In [ ]:
# compile model with optimizer setting 2
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss=my_loss_fn, metrics=[binary_acc])

In [ ]:
# coh_loss = 0.1, load weights from coh_loss = 0.5 since run out of GPU time
model.fit(training_generator, validation_data=validation_generator, epochs=1000, callbacks=[MyCustomCallback_44()], verbose=2)

Epoch 1/1000
63/63 [==============================] - 87s 1s/step
F-measure: 0.544 vs 0.544
Error rate: 0.580 vs 0.580
187/187 - 1792s - loss: 17.9932 - binary_acc: 0.9466 - val_loss: 22.7221 - val_binary_acc: 0.9264 - 1792s/epoch - 10s/step
Epoch 2/1000
63/63 [==============================] - 85s 1s/step
F-measure: 0.548 vs 0.548
Error rate: 0.575 vs 0.575
187/187 - 1796s - loss: 17.8268 - binary_acc: 0.9472 - val_loss: 23.3130 - val_binary_acc: 0.9240 - 1796s/epoch - 10s/step
Epoch 3/1000
63/63 [==============================] - 81s 1s/step
F-measure: 0.555 vs 0.555
Error rate: 0.571 vs 0.571
187/187 - 1747s - loss: 17.9094 - binary_acc: 0.9468 - val_loss: 22.3620 - val_binary_acc: 0.9274 - 1747s/epoch - 9s/step
Epoch 4/1000
63/63 [==============================] - 81s 1s/step
F-measure: 0.557 vs 0.557
Error rate: 0.566 vs 0.566
187/187 - 1746s - loss: 17.8882 - binary_acc: 0.9470 - val_loss: 22.8096 - val_binary_acc: 0.9266 - 1746s/epoch - 9s/step
Epoch 5/1000
63/63 [==============

In [ ]:
# coh_loss = 0.5
model.fit(training_generator, validation_data=validation_generator, epochs=1000, callbacks=[MyCustomCallback_44()], verbose=2)

Epoch 1/1000
63/63 [==============================] - 3s 31ms/step
F-measure: nan vs 0.000
Error rate: 1.000 vs 1.000
187/187 - 85s - loss: 83.6491 - binary_acc: 0.9148 - val_loss: 92.8042 - val_binary_acc: 0.9213 - 85s/epoch - 457ms/step
Epoch 2/1000
63/63 [==============================] - 2s 25ms/step
F-measure: nan vs 0.000
Error rate: 1.000 vs 1.000
187/187 - 45s - loss: 62.3356 - binary_acc: 0.9197 - val_loss: 66.6647 - val_binary_acc: 0.9213 - 45s/epoch - 241ms/step
Epoch 3/1000
63/63 [==============================] - 2s 26ms/step
F-measure: nan vs 0.000
Error rate: 1.000 vs 1.000
187/187 - 44s - loss: 49.3521 - binary_acc: 0.9199 - val_loss: 48.2002 - val_binary_acc: 0.9214 - 44s/epoch - 237ms/step
Epoch 4/1000
63/63 [==============================] - 2s 26ms/step
F-measure: 0.002 vs 0.002
Error rate: 0.999 vs 0.999
187/187 - 49s - loss: 41.6526 - binary_acc: 0.9207 - val_loss: 41.9937 - val_binary_acc: 0.9211 - 49s/epoch - 262ms/step
Epoch 5/1000
63/63 [======================

In [ ]:
# coh_loss = 1.5
model.fit(training_generator, validation_data=validation_generator, epochs=1000, callbacks=[MyCustomCallback_44()], verbose=2)

Epoch 1/1000
63/63 [==============================] - 3s 32ms/step
F-measure: nan vs 0.000
Error rate: 1.000 vs 1.000
187/187 - 84s - loss: 83.9003 - binary_acc: 0.9134 - val_loss: 105.1496 - val_binary_acc: 0.9212 - 84s/epoch - 448ms/step
Epoch 2/1000
63/63 [==============================] - 2s 27ms/step
F-measure: nan vs 0.000
Error rate: 1.000 vs 1.000
187/187 - 48s - loss: 61.6494 - binary_acc: 0.9197 - val_loss: 74.1958 - val_binary_acc: 0.9212 - 48s/epoch - 257ms/step
Epoch 3/1000
63/63 [==============================] - 2s 26ms/step
F-measure: nan vs 0.000
Error rate: 1.000 vs 1.000
187/187 - 45s - loss: 48.5595 - binary_acc: 0.9199 - val_loss: 50.2309 - val_binary_acc: 0.9213 - 45s/epoch - 242ms/step
Epoch 4/1000
63/63 [==============================] - 2s 26ms/step
F-measure: 0.001 vs 0.001
Error rate: 0.999 vs 0.999
187/187 - 49s - loss: 40.7893 - binary_acc: 0.9206 - val_loss: 40.4519 - val_binary_acc: 0.9214 - 49s/epoch - 260ms/step
Epoch 5/1000
63/63 [=====================